In [768]:
# survival - Survival (0 = No; 1 = Yes)
#class - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
#name - Name
#sex - Sex
#age - Age
#sibsp - Number of Siblings/Spouses Aboard
#parch - Number of Parents/Children Aboard
#ticket - Ticket Number
#fare - Passenger Fare
#cabin - Cabin
#embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
#boat - Lifeboat (if survived)
#body - Body number (if did not survive and body was recovered)

## Part I: Exploratory analysis (3 points)

Read the dataset correctly using pandas, using an appropriate column as index (1 point)

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv")
train = train.set_index("PassengerId")

train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Compute the percentage of survivors out of total number of passengers...

In [2]:
survivors = train["Survived"].sum()

total_passengers = len(train)

percentage_survivors = round(survivors/total_passengers*100,2)

print(f" The percentage of survivors out of the total number of passengers is {percentage_survivors}%.")

 The percentage of survivors out of the total number of passengers is 38.38%.


...and the percentage of men and women out of total number of passengers (0.5 points)

In [3]:
men = train[train["Sex"] == "male"]["Sex"].count()
women = train[train["Sex"] == "female"]["Sex"].count()

percentage_men = round(men/total_passengers*100,2)
percentage_women = round(women/total_passengers*100,2)

print(f" The percentage of men is {percentage_men}% and of women {percentage_women}%.")

 The percentage of men is 64.76% and of women 35.24%.


Compute the percentage of survivors by sex (i.e. the percentage of male passengers that survived and female passengers that survived)...

In [4]:
men_survived = train[(train["Sex"] == "male") 
                     & (train["Survived"] == 1)]["Survived"].count()

women_survived = train[(train["Sex"] == "female") 
                       & (train["Survived"] == 1)]["Survived"].count()

perc_men_surv = round(men_survived/men*100,2)
perc_women_surv = round(women_survived/women*100,2)

print(f" The percentage of men survived is {perc_men_surv}% and of women survived {perc_women_surv}%.")

 The percentage of men survived is 18.89% and of women survived 74.2%.


...and the sex distribution of survivors (i.e. percentage of survivors that were men and percentage of survivors that were women) (0.5 points)

In [5]:
perc_men_of_surv = round(men_survived/survivors*100,2)
perc_women_of_surv = round(women_survived/survivors*100,2)

print(f" The percentage of men survivors is {perc_men_of_surv}% and of women survivors {perc_women_of_surv}%.")

 The percentage of men survivors is 31.87% and of women survivors 68.13%.


Display in a 2 x 2 DataFrame the probability of being male/female and surviving/not surviving (0.5 points)

One possible option:

|            | **Survived**      | **Not survived**      |
|------------|-------------------|-----------------------|
| **Male**   | Male & Survived   | Male & Not survived   |
| **Female** | Female & Survived | Female & Not survived |

Notice that the sum of all values in the table above should be 1 (or 100 %).

In [7]:
round(train.pivot_table(values = "Name", 
                        index = "Sex", 
                        aggfunc = "count", 
                        columns = "Survived")
.div(len(train)) * 100,2).astype(str) + '%'

# At the end we put .astype(str)+'%' because now after each value we see the '%'

Survived,0,1
Sex,,
female,9.09%,26.15%
male,52.53%,12.23%


Display in a DataFrame the probability of survival/not survival of all combinations of sex and class (0.5 points)

One possible option:

|            |   | **Survived**              | **Not survived** |
|------------|---|---------------------------|------------------|
| **Male**   | 1 | Male & Survived & Class 1 | ...              |
|            | 2 | Male & Survived & Class 2 | ...              |
|            | 3 | Male & Survived & Class 3 | ...              |
| **Female** | 1 | ...                       | ...              |
|            | 2 | ...                       | ...              |
|            | 3 | ...                       | ...              |

Notice that the sum of all values in the table above should be 1 (or 100 %).

In [ ]:
# We put "Name" in the function because these are all unique values ("PassengerId" would also be possible if this
# was not the index)

round(train.pivot_table(values = "Name", 
                        index = ["Sex","Pclass"], 
                        aggfunc = "count", 
                        columns = "Survived")
.div(len(train))*100,2)
.astype(str)+'%'

## Part II: More insights on the data (3 points)

Present 3 insights about the dataset, each of them relating at least 3 different variables, and support them by code and numbers. Possible examples:

- "**Men** aged **less than 18** were more/less likely to **survive** than the average passenger" (Sex, Age, Survival)
- "**Women** with **no siblings or spouses** paid on average a cheaper/pricier **ticket** than the average woman" (Sex, SibSp, Fare)
- "**Men** with a **title other than Mr.** were more/less likely to have a known (i.e. non-missing) **cabin** than the average man" (Sex, Name, Cabin)

(Using these exact examples is valid, but awards fewer points than proposing original insights)

_Hint: If you want to work with lists and dictionaries rather than pandas objects, you can do_

```python
recs = df.to_dict(orient="records")
```

In [ ]:
# 1ST INSIGHT: CABIN & PCLASS --> SURVIVAL RATE

In [779]:
df_1 = train.pivot_table(values = "Name", 
                         index = [train["Cabin"].astype(str).str[0], "Pclass"],
                         aggfunc = "count",
                         columns = "Survived")

df_1 = round(df_1.div(df_1.sum(axis=1), axis = 0)*100,2).astype(str) + "%"

# We did this division because now we get the relative percentages per class, otherwise you cannot compare different
# cabins

df_1.rename(columns = {0 : "Not Survived", 1 : "Survived"})

Survived     Not Survived Survived
Cabin Pclass                      
A     1            53.33%   46.67%
B     1            25.53%   74.47%
C     1            40.68%   59.32%
D     1            24.14%   75.86%
      2             25.0%    75.0%
E     1             28.0%    72.0%
      2             25.0%    75.0%
      3              nan%   100.0%
F     2             12.5%    87.5%
      3             80.0%    20.0%
G     3             50.0%    50.0%
T     1            100.0%     nan%
n     1             52.5%    47.5%
      2            55.95%   44.05%
      3            76.41%   23.59%

In [ ]:
# High class cabins: A, B, C, D
# Low class cabins: E, F, G

# Best survival rate: B, D, E, F(2)
# Worst survival rate: A, F(3), T, NaN

In [ ]:
# 2ND INSIGHT: TITLE & AGE GROUP 

In [776]:
# Extracting Title from the Name column and storing the results in a new column Title

train["Title"] = train["Name"].str.extract("([A-Za-z]+)\.") # We take capital (A-Z) followed by small letter (a-z) 

age_category = pd.cut(train.Age,
                      bins=[min(train["Age"]), 5, 18, 45, max(train["Age"])],   # make categories of age
                      labels=['Baby','Child','Adult','Old'])

train.insert(9,'Age Group',age_category) # insert the categories in dataframe

In [777]:
dic1 = dict(train["Name"].groupby(train["Title"]).count()) # group the passengers by their titles by counting them
dic2 = {}

for key, value in dic1.items(): # if a title only appears once in the dataset, put it in a new dataset
    if value == 1:
        dic2.update({key:value})
        
rares = [key for key,value in dic2.items()] 
train["Title"] = train["Title"].replace(rares,"Rare") # We call these titles as 'rare'

In [778]:
# Make a pivot table of the titles, age groups and their survival rate

df_2 = round(train.pivot_table(values = "Name",
                             index = ["Title","Age Group"],
                             aggfunc = "count",
                             columns = ["Survived"]).div(len(train))*100,2) 

df_2 = df_2[(df_2[0] != 0) & (df_2[1] != 0)].astype(str) + '%' # Delete those rows where both values are 0 
# (to make the dataframe more compact)

df_2.rename(columns = {0 : "Not Survived", 1 : "Survived"})

Survived         Not Survived Survived
Title  Age Group                      
Col    Old              0.11%    0.11%
Dr     Adult            0.22%    0.11%
       Old              0.11%    0.22%
Master Baby              0.9%    1.57%
       Child            0.79%    0.67%
Miss   Baby             0.56%     1.8%
       Child             1.8%    2.69%
       Adult            2.13%    6.62%
       Old              0.11%    0.67%
Mr     Child            3.59%    0.34%
       Adult           27.38%    6.29%
       Old              6.17%     0.9%
Mrs    Child            0.11%    0.67%
       Adult            2.13%    6.85%
       Old              0.34%    2.02%
Rare   Adult            0.22%    0.34%
       Old              0.11%    0.22%

In [ ]:
# Colonels are old people
# Masters are children/babies
# Rares and doctors are adults/olds

# Adult/old misses and mrs best survival rate
# Adult/old misters and mister children worst survival rate

In [ ]:
# 3RD INSIGHT: Fare Grouping

In [780]:
[q1,q2,q3] = train["Fare"].quantile([0.25,0.5,0.75]) # Make quartiles of Fare to optimize your bins 
# (similar amount of passengers in eacht bin)

# Now make categories of Fare, using these bins: 'Very Low','Low','High','Very High'
category = pd.cut(train.Fare,bins=[train["Fare"].min(),q1,q2,q3,train["Fare"].max()],
                  labels=['Very Low','Low','High','Very High'])

# Insert these categories in the dataframe
train.insert(9,'Fare Group',category)

In [781]:
# Make a pivot table of: Sex, Fare Group and their survival rate
df3 = round(train.pivot_table(values = "Name", 
                              index = ["Sex","Fare Group"], 
                              aggfunc = "count", 
                              columns = "Survived")
      .div(len(train))*100,2).astype(str)+'%'

df3.rename(columns = {0 : "Not Survived", 1 : "Survived"})

# Because the bins are already optimized with the quantiles, these percentages are already balanced and we don't
# need the relative percentages

Survived          Not Survived Survived
Sex    Fare Group                      
female Very Low          1.46%    3.37%
       Low               2.69%    4.83%
       High              3.14%     7.3%
       Very High          1.8%   10.66%
male   Very Low         17.06%    1.46%
       Low              14.81%    2.81%
       High             10.44%    4.04%
       Very High         8.64%    3.82%

In [ ]:
# best surival rate: very high paying women
# worst survival rate: very low paying men